In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import re, sys, operator
from pyspark import SparkConf, SparkContext
conf = SparkConf()
sc = SparkContext(conf=conf)

In [3]:
line = sc.textFile('./browsing.txt')
baskets = line.map(lambda m: m.split()).map(lambda m: sorted(set(m)))
baskets.take(5)

[['ELE17451', 'ELE89019', 'FRO11987', 'GRO99222', 'SNA90258'],
 ['ELE17451',
  'ELE26917',
  'ELE52966',
  'ELE91550',
  'FRO12685',
  'FRO84225',
  'FRO90334',
  'GRO12298',
  'GRO99222',
  'SNA11465',
  'SNA30755',
  'SNA80192'],
 ['DAI22896', 'ELE17451', 'FRO86643', 'GRO73461', 'SNA99873'],
 ['ELE17451', 'ELE23393', 'ELE37798', 'FRO86643', 'GRO56989', 'SNA11465'],
 ['DAI54444',
  'ELE11375',
  'ELE17451',
  'ELE28573',
  'FRO78087',
  'FRO86643',
  'GRO39357',
  'SNA11465',
  'SNA69641']]

QUESTION 2D: FOR DOUBLES

In [4]:
def calculate_single_items(basket):
    single_items = []
    for item in basket:
        single_items.append((item, 1))
    return single_items

single_items_support = (baskets.flatMap(calculate_single_items).reduceByKey(operator.add).filter(lambda m: m[1] >= 100))
single_items_dict = {}
for item, support in single_items_support.collect():
    single_items_dict[item] = support
single_items_dict = sc.broadcast(single_items_dict)

def calculate_double_items(basket):
    singles = single_items_dict.value
    double_items = []
    for i in range(len(basket)):
        if basket[i] in singles:
            for j in range(i):
                if basket[j] in singles:
                    double_items.append(((basket[j], basket[i]), 1))
    return double_items

double_items_support = (baskets.flatMap(calculate_double_items).reduceByKey(operator.add).filter(lambda m: m[1] >= 100))
def calculate_conf_double_items(double_items_support):
    double, support = double_items_support
    support = float(support)
    x, y = double
    singles = single_items_dict.value
    conf_xy = support / singles[x]
    conf_yx = support / singles[y]
    return (('%s -> %s' % (x, y), conf_xy),
            ('%s -> %s' % (y, x), conf_yx))

conf_double_items = double_items_support.flatMap(calculate_conf_double_items).sortBy(lambda m: (-m[1], m[0]))
double_items_dict = {}
for item, support in double_items_support.collect():
    double_items_dict[item] = support

print("Top 5 doubles:")
double_items_dict = sc.broadcast(double_items_dict)
conf_double_items.take(5)

Top 5 doubles:


[('DAI93865 -> FRO40251', 1.0),
 ('GRO85051 -> FRO40251', 0.999176276771005),
 ('GRO38636 -> FRO40251', 0.9906542056074766),
 ('ELE12951 -> FRO40251', 0.9905660377358491),
 ('DAI88079 -> FRO40251', 0.9867256637168141)]

QUESTION 2E: FOR TRIPLES

In [5]:
def calculate_triple_items(basket):
    doubles = double_items_dict.value
    singles = single_items_dict.value
    triple_items = []
    for i in range(len(basket)):
        if basket[i] not in singles:
            continue
        for j in range(i):
            if basket[j] not in singles:
                continue
            if (basket[j], basket[i]) not in doubles:
                continue
            for k in range(j):
                if basket[k] not in singles:
                    continue
                if (basket[k], basket[j]) not in doubles:
                    continue
                if (basket[k], basket[i]) not in doubles:
                    continue
                triple_items.append(((basket[k], basket[j], basket[i]), 1))
    return triple_items


triple_items_support = (baskets.flatMap(calculate_triple_items).reduceByKey(operator.add).filter(lambda m: m[1] >= 100))

def calculate_conf_triple_items(triple_items_support):
    doubles = double_items_dict.value
    triple, support = triple_items_support
    support = float(support)
    x, y, z = triple
    conf_xy_z = support / doubles[x, y]
    conf_xz_y = support / doubles[x, z]
    conf_yz_x = support / doubles[y, z]
    return (('(%s, %s) -> %s' % (x, y, z), conf_xy_z),
            ('(%s, %s) -> %s' % (x, z, y), conf_xz_y),
            ('(%s, %s) -> %s' % (y, z, x), conf_yz_x))
print("Top 5 triples:")
conf_triple_items = triple_items_support.flatMap(calculate_conf_triple_items).sortBy(lambda m: (-m[1], m[0]))
conf_triple_items.take(5)

Top 5 triples:


[('(DAI23334, ELE92920) -> DAI62779', 1.0),
 ('(DAI31081, GRO85051) -> FRO40251', 1.0),
 ('(DAI55911, GRO85051) -> FRO40251', 1.0),
 ('(DAI62779, DAI88079) -> FRO40251', 1.0),
 ('(DAI75645, GRO85051) -> FRO40251', 1.0)]